# Facical Expression Detection
This model generated by ChatGPT to do expression analysis of faces for dataset from Kaggle (size 48x48 and grayscale)


Add the required libraries

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import requests
from sklearn.model_selection import train_test_split

Download the dataset and validate it

In [7]:
# Download the dataset
#url = "https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/download/fer2013.csv"
filename = "fer2013.csv"
#response = requests.get(url)
#open(filename, "wb").write(response.content)

# Load the dataset
data = pd.read_csv(filename)

# Get the unique emotions in the "emotion" column
emotions = data['emotion'].unique()
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Map emotions to integers
emotion_mapping = {emotion: i for i, emotion in enumerate(emotion_labels)}

# Print the mapping
print(emotion_mapping)
data.head()

{'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3, 'Sad': 4, 'Surprise': 5, 'Neutral': 6}


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


Data transformation and prepare the data as training and testing datasets (20% testing data)

In [ ]:
# Split the strings in the pixels column into a list of integers
pixels = data['pixels'].str.split().apply(lambda x: np.array(x).astype(int))

# Convert the list of integers to a numpy array
images = np.vstack(pixels.values)

# Reshape the array as needed
images = np.reshape(images, (-1, 48, 48, 1))

# Reshape the array as needed
#images = np.reshape(images, (-1, 48, 48, 1))
#images = np.array(data['pixels'].tolist(), dtype=np.uint8)
#images = np.reshape(images, (-1, 48, 48, 1))
labels = tf.keras.utils.to_categorical(data['emotion'], 7)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

print(len(X_train))
print(len(X_test))


In [4]:
# Define the model
# Define the model using the Sequential API from TensorFlow's Keras library
model = tf.keras.models.Sequential()

# Add the first Conv2D layer to the model
# 32 filters of size (3, 3) with a ReLU activation function are applied to the input image
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))

# Add a MaxPooling2D layer to the model
# The layer will reduce the size of the feature maps by half after the Conv2D layer
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Add the second Conv2D layer to the model
# 64 filters of size (3, 3) with a ReLU activation function are applied to the output of the previous layer
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))

# Add another MaxPooling2D layer to the model
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Add the third Conv2D layer to the model
# 128 filters of size (3, 3) with a ReLU activation function are applied to the output of the previous layer
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))

# Add a final MaxPooling2D layer to the model
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Add a Flatten layer to the model
# This layer will convert the 3D tensor output of the Conv2D layers into a 1D vector
model.add(tf.keras.layers.Flatten())

# Add a dense layer to the model with 128 units and a ReLU activation function
model.add(tf.keras.layers.Dense(128, activation='relu'))

# Add a dropout layer to the model with a rate of 0.5
# This layer will randomly set 50% of the inputs to 0 during training to prevent overfitting
model.add(tf.keras.layers.Dropout(0.5))

# Add the final dense layer to the model with 7 units and a softmax activation function
# This layer will provide the 7 class predictions for the emotion recognition task
model.add(tf.keras.layers.Dense(7, activation='softmax'))


## Model Compilation
The compile method is used to compile the model before training it. The optimizer argument specifies the optimization algorithm to be used during training, in this case 'adam' which is a popular stochastic gradient-based optimization algorithm. The loss argument defines the loss function to be used during training, which in this case is 'categorical_crossentropy'. This loss function is used for multi-class classification problems and measures the difference between predicted class probabilities and true labels. The metrics argument specifies the evaluation metric to be used to assess the model performance during training and testing, in this case 'accuracy'.

In [5]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


## Model Training
The following line trains the model on the training data X_train and the corresponding labels y_train
The training process will run for a total of 10 epochs
The batch size of 32 means the model will process 32 samples at a time before updating the model weights
During the training process, the model will be evaluated on the validation data (X_test, y_test) and these results will be used to monitor the model's performance and prevent overfitting.
The results of the training process will be stored in the history object, which can be used to visualize the model's training and validation accuracy/loss over time.

In [7]:
# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
898/898 [==============================] - 74s 82ms/step - loss: 1.8963 - accuracy: 0.2791 - val_loss: 1.6268 - val_accuracy: 0.3692
Epoch 2/10
898/898 [==============================] - 73s 81ms/step - loss: 1.5972 - accuracy: 0.3792 - val_loss: 1.4886 - val_accuracy: 0.4257
Epoch 3/10
898/898 [==============================] - 73s 81ms/step - loss: 1.5043 - accuracy: 0.4180 - val_loss: 1.4559 - val_accuracy: 0.4349
Epoch 4/10
898/898 [==============================] - 73s 82ms/step - loss: 1.4426 - accuracy: 0.4420 - val_loss: 1.3952 - val_accuracy: 0.4671
Epoch 5/10
898/898 [==============================] - 72s 80ms/step - loss: 1.3939 - accuracy: 0.4648 - val_loss: 1.3749 - val_accuracy: 0.4738
Epoch 6/10
898/898 [==============================] - 72s 81ms/step - loss: 1.3488 - accuracy: 0.4828 - val_loss: 1.3236 - val_accuracy: 0.4922
Epoch 7/10
898/898 [==============================] - 72s 81ms/step - loss: 1.3152 - accuracy: 0.4936 - val_loss: 1.3026 - val_accuracy:

## Model Evaluation
It will computes the loss and accuracy metrics for the test data. The input parameters for this method are X_test and y_test which represent the test data and target labels respectively. The verbose parameter is used to specify the level of verbosity. Setting verbose=0 means no output will be displayed during the evaluation process.
The method returns two values, test_loss and test_acc, which represent the loss and accuracy metrics respectively. These values can be used to determine how well the model is performing on the test data.

In [8]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', test_acc)



Test accuracy: 0.5084981918334961


## Save the model
It will uses the HDF5 (Hierarchical Data Format version 5). It is a file format for storing large amounts of data in a binary file format that can be efficiently read and written using Python and other programming languages. The extension .h5 indicates that the model is saved in HDF5 format.

In [9]:
# Save the model
model.save('facial_expression_recognition_model.h5')